In [ ]:
!pip install igraph
!pip install scikit-image==0.20.0

In [2]:
from GNEMS.GNEMS import GNEMS_segment
from PIL import Image
from PIL.PngImagePlugin import PngInfo
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.color import label2rgb
from glob import glob
from tqdm import tqdm
import time
import threading

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [1]:
!nvidia-smi

Sun Apr  2 22:14:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          On   | 00000000:06:00.0 Off |                    0 |
|  0%   65C    P0    87W / 150W |      0MiB / 23028MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
method_name = f"GNEMS_alternate_{time.time()}"
!mkdir results/{method_name}
!mkdir results/{method_name}/noise
!mkdir results/{method_name}/noise/0.5
!mkdir results/{method_name}/noise/1.0
!mkdir results/{method_name}/noise/2.0
!mkdir results/{method_name}/noise/4.0
!mkdir results/{method_name}/noise/8.0
!mkdir results/{method_name}/clouds
!mkdir results/{method_name}/texture

def segment_and_save(path, id, bar=None):
    im = np.array(Image.open(path).convert("L"))
    d = 16
    seg = GNEMS_segment(im, d=d, iterations=50, prediction_stride=8, show_progress=False)
    category = path.split("/")[1]
    Image.fromarray(seg).convert("L").save(f"results/{method_name}/{category}/{id}.png")
    bar.update(1)

print("Segmenting clouds dataset...")
with open("datasets/clouds/test_ids.txt", "r") as f:
    test_ids = [line.strip() for line in f.readlines()]

threads = []
with tqdm(total = len(test_ids)) as pbar:
    for id in test_ids:
        path = f"datasets/clouds/images/{id}.png"
        t = threading.Thread(target=segment_and_save, args=(path,id,pbar))
        while len(threading.enumerate()) >= 64:
            time.sleep(1)
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

with open("datasets/noise/test_ids.txt", "r") as f:
    test_ids = [line.strip() for line in f.readlines()]

print("Segmenting noise dataset...")
threads = []
with tqdm(total = len(test_ids)) as pbar:
    for id in test_ids:
        path = f"datasets/noise/{id}.png"
        t = threading.Thread(target=segment_and_save, args=(path,id,pbar))
        while len(threading.enumerate()) >= 64:
            time.sleep(1)
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

print("Segmenting texture dataset...")
with open("datasets/texture/test_ids.txt", "r") as f:
    test_ids = [line.strip() for line in f.readlines()]

threads = []
with tqdm(total = len(test_ids)) as pbar:
    for id in test_ids:
        path = f"datasets/texture/images/{id}.png"
        t = threading.Thread(target=segment_and_save, args=(path,id,pbar))
        while len(threading.enumerate()) >= 64:
            time.sleep(1)
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

Segmenting clouds dataset...


100%|██████████| 256/256 [00:52<00:00,  4.88it/s]


Segmenting noise dataset...


100%|██████████| 1280/1280 [04:20<00:00,  4.92it/s]


Segmenting texture dataset...


100%|██████████| 768/768 [02:35<00:00,  4.94it/s]


In [4]:
print(method_name)

GNEMS_alternate_1680473732.0023267
